In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm.auto import tqdm
import warnings

warnings.filterwarnings('ignore')
tqdm.pandas()

DATA_PATH = '/kaggle/input/nto-ai-second-stage-individual/' # Укажите путь к вашим данным
RANDOM_STATE = 42

In [ ]:
print("Загрузка всех файлов")
train_df = pd.read_csv(f'{DATA_PATH}train.csv', sep=',')
test_df = pd.read_csv(f'{DATA_PATH}test.csv', sep=',')
users_df = pd.read_csv(f'{DATA_PATH}users.csv', sep=',')
book_genres_df = pd.read_csv(f'{DATA_PATH}book_genres.csv', sep=',')

books_df = pd.read_csv(f'{DATA_PATH}books.csv', sep=',', engine='python', encoding='utf-8-sig', on_bad_lines='skip')
book_descriptions_df = pd.read_csv(f'{DATA_PATH}book_descriptions.csv', sep=',', engine='python', encoding='utf-8-sig', on_bad_lines='skip')

for df in [train_df, test_df, users_df, books_df, book_genres_df, book_descriptions_df]:
    df.columns = df.columns.str.strip()

train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])

print("\nДанные загружены")
print("\nПроверка колонок в books_df:")
print(list(books_df.columns))

In [ ]:
print("Объединение таблиц")

train_df = train_df.merge(users_df, on='user_id', how='left')
train_df = train_df.merge(books_df, on='book_id', how='left')

test_df = test_df.merge(users_df, on='user_id', how='left')
test_df = test_df.merge(books_df, on='book_id', how='left')

print("\nДанные успешно объединены")
train_df.head()

In [ ]:
train_read_df = train_df[train_df['has_read'] == 1].copy()
train_read_df = train_read_df.sort_values('timestamp').reset_index(drop=True)

print(f"Всего оценок в трейне: {len(train_read_df)}")

split_index = int(len(train_read_df) * 0.9)
train_part = train_read_df.iloc[:split_index]
val_part = train_read_df.iloc[split_index:]

print(f"Размер обучающей части: {len(train_part)}")
print(f"Размер валидационной части: {len(val_part)}")

assert train_part['timestamp'].max() < val_part['timestamp'].min()
print("Временная валидация настроена корректно")

In [ ]:
def create_features(df, train_data):
    result_df = df.copy()

    global_mean_rating = train_data['rating'].mean()

    user_stats = train_data.groupby('user_id')['rating'].agg(['mean', 'count', 'std']).reset_index()
    user_stats.columns = ['user_id', 'user_mean_rating', 'user_ratings_count', 'user_std_rating']
    
    result_df = result_df.merge(user_stats, on='user_id', how='left')

    item_stats = train_data.groupby('book_id')['rating'].agg(['mean', 'count', 'std']).reset_index()
    item_stats.columns = ['book_id', 'book_mean_rating', 'book_ratings_count', 'book_std_rating']
    
    result_df = result_df.merge(item_stats, on='book_id', how='left')

    result_df['user_rating_diff'] = result_df['user_mean_rating'] - global_mean_rating
    result_df['book_rating_diff'] = result_df['book_mean_rating'] - global_mean_rating

    result_df.fillna(global_mean_rating, inplace=True)
    
    return result_df

train_featured = create_features(train_part, train_part)
val_featured = create_features(val_part, train_part)
test_featured = create_features(test_df, train_read_df)

print("Признаки сгенерированы.")
display(train_featured.head())

In [ ]:
features = [
    'age', 'gender', 'publication_year', 'avg_rating',
    'user_mean_rating', 'user_ratings_count', 'user_std_rating',
    'book_mean_rating', 'book_ratings_count', 'book_std_rating',
    'user_rating_diff', 'book_rating_diff'
]
target = 'rating'

X_train = train_featured[features]
y_train = train_featured[target]
X_val = val_featured[features]
y_val = val_featured[target]

lgb_params = {
    'objective': 'regression_l1', 
    'metric': 'rmse',
    'n_estimators': 2000,
    'learning_rate': 0.06,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'num_leaves': 31,
    'verbose': -1,
    'n_jobs': -1,
    'seed': RANDOM_STATE,
    'boosting_type': 'gbdt',
}

model = lgb.LGBMRegressor(**lgb_params)

model.fit(X_train, y_train,
          eval_set=[(X_val, y_val)],
          eval_metric='rmse',
          callbacks=[lgb.early_stopping(100, verbose=True)])

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm.auto import tqdm
import random

# --- НАСТРОЙКИ ---
TARGET_RMSE = 2.68781
TARGET_MAE = 1.89   # (Для справки, фильтровать будем по RMSE как основной метрике)
DESIRED_SEEDS_COUNT = 50 
MAX_ATTEMPTS = 10000       # Защита от бесконечного цикла (если 42 был слишком хорош)

lgb_params_base = {
    'objective': 'regression_l1', 
    'metric': 'rmse',
    'n_estimators': 3000,
    'learning_rate': 0.06,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'num_leaves': 31,
    'verbose': -1,
    'n_jobs': -1,
    'seed': RANDOM_STATE,
    'boosting_type': 'gbdt',
}

# --- Хранилища ---
good_seeds = []
test_preds_accum = np.zeros(len(test_featured))
val_preds_accum = np.zeros(len(X_val))

print(f"🎰 ЗАПУСКАЕМ КАЗИНО! Цель: найти {DESIRED_SEEDS_COUNT} сидов с RMSE < {TARGET_RMSE}")

pbar = tqdm(total=DESIRED_SEEDS_COUNT)
attempts = 0

while len(good_seeds) < DESIRED_SEEDS_COUNT and attempts < MAX_ATTEMPTS:
    attempts += 1

    current_seed = np.random.randint(1, 100000)
    if current_seed == 42 or current_seed in good_seeds:
        continue

    params = lgb_params_base.copy()
    params['random_state'] = current_seed
    
    model = lgb.LGBMRegressor(**params)
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='rmse',
        callbacks=[lgb.early_stopping(100, verbose=False)]
    )

    val_pred = model.predict(X_val)
    val_pred = np.clip(val_pred, 0, 10)
    rmse = np.sqrt(mean_squared_error(y_val, val_pred))

    if rmse < TARGET_RMSE:
        print(f"💎 НАЙДЕН АЛМАЗ! Seed: {current_seed} | RMSE: {rmse:.5f} (Better by {TARGET_RMSE - rmse:.5f})")

        t_pred = model.predict(test_featured[features])
        t_pred = np.clip(t_pred, 0, 10)
        
        test_preds_accum += t_pred
        val_preds_accum += val_pred
        good_seeds.append(current_seed)
        pbar.update(1)
    else:
        if attempts % 10 == 0:
            print(f"Attempt {attempts}: Seed {current_seed} gave RMSE {rmse:.5f} (Too bad)")

pbar.close()

# --- ФИНАЛ ---
if len(good_seeds) > 0:
    print(f"\n✅ Собрано {len(good_seeds)} сидов: {good_seeds}")
    
    # Усредняем
    final_test_preds = test_preds_accum / len(good_seeds)
    final_val_preds = val_preds_accum / len(good_seeds)
    
    # Чек метрики ансамбля
    ensemble_rmse = np.sqrt(mean_squared_error(y_val, final_val_preds))
    print(f"\n🏆 RMSE АНСАМБЛЯ: {ensemble_rmse:.5f}")
    if ensemble_rmse < TARGET_RMSE:
        print(f"🚀 ПРОБОЙ! Ансамбль лучше одиночной модели на {TARGET_RMSE - ensemble_rmse:.5f}")
    else:
        print("🤔 Ансамбль не побил лучшую одиночную (бывает, если сиды слишком коррелируют)")

    # Сабмит
    submission = pd.DataFrame({
        'user_id': test_df['user_id'],
        'book_id': test_df['book_id'],
        'rating_predict': final_test_preds
    })
    
    filename = 'submission_casino_royale_final_pupupu.csv'
    submission.to_csv(filename, index=False)
    print(f"Файл {filename} готов.")
    
else:
    print("❌ Не найдено ни одного сида лучше 42. Попробуй смягчить условие или увеличь learning_rate.")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))

lgb.plot_importance(model, max_num_features=20, importance_type='gain', figsize=(12, 8), title='Feature Importance (Gain)')
plt.show()

lgb.plot_importance(model, max_num_features=20, importance_type='split', figsize=(12, 8), title='Feature Importance (Split)')
plt.show()

In [ ]:
val_preds = model.predict(X_val)

val_preds = np.clip(val_preds, 0, 10)

rmse = np.sqrt(mean_squared_error(y_val, val_preds))
mae = mean_absolute_error(y_val, val_preds)

print(f"Локальная валидация:")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

In [ ]:
print("Обучение финальной модели на всех данных")

X_full = pd.concat([train_featured[features], val_featured[features]])
y_full = pd.concat([train_featured[target], val_featured[target]])

best_iteration = model.best_iteration_
if best_iteration is None:
    best_iteration = lgb_params['n_estimators']
    
lgb_params['n_estimators'] = best_iteration

final_model = lgb.LGBMRegressor(**lgb_params)
final_model.fit(X_full, y_full)

print("Предсказание для тестовых данных...")
X_test = test_featured[features]
test_preds = final_model.predict(X_test)

test_preds = np.clip(test_preds, 0, 10)

submission_df = pd.DataFrame({
    'user_id': test_df['user_id'],
    'book_id': test_df['book_id'],
    'rating_predict': test_preds
})

submission_df.to_csv('submission_final_pupupu.csv', index=False, sep=',')
print("Файл submission.csv успешно создан.")
display(submission_df.head())